In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/val/val_week1_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/test/test_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/clean_data_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/val/val_week1_2_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/test/test_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/clean_data_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw

## 1. Import libraries

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support

<ipython-input-2-cd41008c27ef>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


## 2. Load dataset

In [3]:
five_fold_files = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_5.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_5.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/5-folds/data_part_5.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week4/train/5-folds/data_part_5.csv"
    ]
}

file_validataion = {
    "week1": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/val/val_week1.csv",
    "week2": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/val/val_week1_2.csv",
    "week3": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/val/val_week1_2_3.csv",
    "week4": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/val/val_week1_2_3_4.csv"
}

file_test = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/test/test_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/test/test_week2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/test/test_week3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/test/test_week4.csv"
    ]
}

In [4]:
# Danh sách tham số tốt nhất (bổ sung)
best_params = {
    "week1": {
        "units_1": 224,
        "dropout_1": 0.4,
        "units_2": 128,
        "dropout_2": 0.3,
        "learning_rate": 0.0021299066766227647
    },
    "week2": {
        "units_1": 160,
        "dropout_1": 0.5,
        "units_2": 160,
        "dropout_2": 0.5,
        "learning_rate": 0.0011567819399700755
    },
    "week3": {
        "units_1": 256,
        "dropout_1": 0.1,
        "units_2": 96,
        "dropout_2": 0.5,
        "learning_rate": 0.0008151332617212836
    },
    "week4": {
        "units_1": 160,
        "dropout_1": 0.1,
        "units_2": 160,
        "dropout_2": 0.2,
        "learning_rate": 0.0007558360818294099
    }
}


## Mô hình BiLSTM

## Dự đoán các week  với best parameters

In [5]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_Bilstm_model(params):
    model = tf.keras.Sequential()
        
    # Bidirectional LSTM layer 1
    model.add(layers.Bidirectional(layers.LSTM(
        units=params.get('units_1'),
        return_sequences=True
    ), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))
    
    # Bidirectional LSTM layer 2
    model.add(layers.Bidirectional(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=False
    )))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))
    
    # Lớp đầu ra
    model.add(layers.Dense(5, activation='softmax'))
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy'])
    return model


In [6]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week]
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Xây dựng mô hình với tham số tốt nhất
        model = build_Bilstm_model(params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["confusion_matrices"].append(conf_matrix)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.mean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.mean(week_results["f1_score_per_label"], axis=0)
    average_confusion_matrix = np.mean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    
    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    
    # In kết quả
    print("\n=== Average Precision, Recall, F1-Score per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)

    # Lưu kết quả tuần vào kết quả tổng quát
    week_results["average_train_times"] = average_train_time
    week_results["average_test_times"] = average_test_time
    week_results["average_metrics_df"] = metrics_df
    week_results["average_confusion_matrix"] = confusion_df
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 224, 'dropout_1': 0.4, 'units_2': 128, 'dropout_2': 0.3, 'learning_rate': 0.0021299066766227647}
Fold 1: Using file /kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week1/train/5-folds/data_part_1.csv as test set


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.6641 - loss: 0.1257 - val_accuracy: 0.7312 - val_loss: 0.1038
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6942 - loss: 0.1143 - val_accuracy: 0.6904 - val_loss: 0.1055
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7017 - loss: 0.1094 - val_accuracy: 0.7148 - val_loss: 0.1028
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7054 - loss: 0.1091 - val_accuracy: 0.7316 - val_loss: 0.1026
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7095 - loss: 0.1118 - val_accuracy: 0.7320 - val_loss: 0.1014
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7033 - loss: 0.1089 - val_accuracy: 0.7003 - val_loss: 0.1022
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7048 - loss: 0.1086 - val_accuracy: 0.7408 - val_loss: 0.0980
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7151 - loss: 0.1059 - val_accuracy: 0

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6635 - loss: 0.1220 - val_accuracy: 0.7007 - val_loss: 0.1118
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7008 - loss: 0.1129 - val_accuracy: 0.7167 - val_loss: 0.1053
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6947 - loss: 0.1137 - val_accuracy: 0.7377 - val_loss: 0.1047
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7024 - loss: 0.1107 - val_accuracy: 0.7388 - val_loss: 0.1057
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7015 - loss: 0.1101 - val_accuracy: 0.7286 - val_loss: 0.1049
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7244 - loss: 0.1069 - val_accuracy: 0.7270 - val_loss: 0.1063
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7092 - loss: 0.1068 - val_accuracy: 0.7510 - val_loss: 0.0991
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7011 - loss: 0.1090 - val_accuracy: 0.7469 - val_

...

328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6914 - loss: 0.1219 - val_accuracy: 0.7689 - val_loss: 0.0934
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7662 - loss: 0.0922 - val_accuracy: 0.7818 - val_loss: 0.0890
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7929 - loss: 0.0841 - val_accuracy: 0.7830 - val_loss: 0.0889
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7865 - loss: 0.0840 - val_accuracy: 0.7963 - val_loss: 0.0847
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7865 - loss: 0.0821 - val_accuracy: 0.7853 - val_loss: 0.0833
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7926 - loss: 0.0804 - val_accuracy: 0.7937 - val_loss: 0.0838
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7952 - loss: 0.0789 - val_accuracy: 0.7937 - val_loss: 0.0820
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7997 - loss: 0.0746 - val_accuracy: 0.7891 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6989 - loss: 0.1191 - val_accuracy: 0.7620 - val_loss: 0.0919
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7709 - loss: 0.0903 - val_accuracy: 0.7895 - val_loss: 0.0863
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7786 - loss: 0.0860 - val_accuracy: 0.7994 - val_loss: 0.0812
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7915 - loss: 0.0805 - val_accuracy: 0.7895 - val_loss: 0.0844
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7869 - loss: 0.0807 - val_accuracy: 0.8028 - val_loss: 0.0796
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7925 - loss: 0.0787 - val_accuracy: 0.7971 - val_loss: 0.0781
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7963 - loss: 0.0749 - val_accuracy: 0.7967 - val_loss: 0.0801
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7974 - loss: 0.0756 - val_accuracy: 0.8093 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6921 - loss: 0.1228 - val_accuracy: 0.7647 - val_loss: 0.0891
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7582 - loss: 0.0950 - val_accuracy: 0.7723 - val_loss: 0.0883
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7748 - loss: 0.0893 - val_accuracy: 0.7887 - val_loss: 0.0806
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7855 - loss: 0.0818 - val_accuracy: 0.7876 - val_loss: 0.0832
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7926 - loss: 0.0811 - val_accuracy: 0.7944 - val_loss: 0.0810
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7908 - loss: 0.0799 - val_accuracy: 0.7876 - val_loss: 0.0790
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7972 - loss: 0.0799 - val_accuracy: 0.7879 - val_loss: 0.0806
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7883 - loss: 0.0786 - val_accuracy: 0.7971 - val_

## Kết quả 5 folds

In [7]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_confusion_matrix = week_result["average_confusion_matrix"]

    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print("\nAverage Precision, Recall, F1-Score per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Train Time: 130.6997 seconds
Average Test Time: 0.7772 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.726188        0.632667          0.675814
1      1           0.625000        0.036468          0.067837
2      2           0.647109        0.414863          0.502552
3      3           0.624939        0.390419          0.466663
4      4           0.792265        0.926656          0.854078

Average Confusion Matrix:
       0    1     2     3       4
0  379.6  1.0  12.6   7.4   199.4
1   22.0  3.2   3.4   3.6    55.4
2   34.0  0.6  68.2   8.4    53.2
3   15.2  0.8   3.6  65.2    82.2
4   71.8  0.4  18.8  26.6  1485.8

=== Results for week2 ===
Average Train Time: 131.0758 seconds
Average Test Time: 0.7846 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.739905        0.685333

## Kết quả trên tập test

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_Bilstm_model(params, input_shape):
    model = tf.keras.Sequential()
        
    # Bidirectional LSTM layer 1
    model.add(layers.Bidirectional(layers.LSTM(
        units=params.get('units_1'),
        return_sequences=True
    ), input_shape=input_shape))
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))
    
    # Bidirectional LSTM layer 2
    model.add(layers.Bidirectional(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=False
    )))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))
    
    # Lớp đầu ra
    model.add(layers.Dense(5, activation='softmax'))
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy'])
    return model


In [9]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"]
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))
    input_shape = (X_train_resampled.shape[1], X_train_resampled.shape[2])
    # Xây dựng mô hình với tham số tốt nhất
    model = build_Bilstm_model(params, input_shape)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    
    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix
    })
    
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [10]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.4936 - loss: 0.1907 - val_accuracy: 0.5261 - val_loss: 0.2396
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5440 - loss: 0.1753 - val_accuracy: 0.6233 - val_loss: 0.1860
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5622 - loss: 0.1705 - val_accuracy: 0.6406 - val_loss: 0.2206
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.5798 - loss: 0.1643 - val_accuracy: 0.5375 - val_loss: 0.2270
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5864 - loss: 0.1617 - val_accuracy: 0.6939 - val_loss: 0.1842
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5957 - loss: 0.1587 - val_accuracy: 0.6528 - val_loss: 0.1861
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5987 - loss: 0.1567 - val_accuracy: 0.6653 - val_loss: 0.1779
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.5960 - loss: 0.1561 - val_accura

In [11]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.5148 - loss: 0.1860 - val_accuracy: 0.5283 - val_loss: 0.2329
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.5861 - loss: 0.1595 - val_accuracy: 0.6500 - val_loss: 0.1823
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6106 - loss: 0.1505 - val_accuracy: 0.6538 - val_loss: 0.1687
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6183 - loss: 0.1450 - val_accuracy: 0.6600 - val_loss: 0.1822
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6250 - loss: 0.1429 - val_accuracy: 0.6231 - val_loss: 0.1912
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.6232 - loss: 0.1428 - val_accuracy: 0.5670 - val_loss: 0.1942
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6363 - loss: 0.1376 - val_accuracy: 0.5360 - val_loss: 0.2181
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6383 - loss: 0.1361 - val_accura

In [12]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.5610 - loss: 0.1689 - val_accuracy: 0.6585 - val_loss: 0.1831
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6518 - loss: 0.1349 - val_accuracy: 0.6942 - val_loss: 0.1508
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6797 - loss: 0.1216 - val_accuracy: 0.7141 - val_loss: 0.1367
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6901 - loss: 0.1162 - val_accuracy: 0.5897 - val_loss: 0.1928
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6974 - loss: 0.1090 - val_accuracy: 0.7111 - val_loss: 0.1264
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6947 - loss: 0.1076 - val_accuracy: 0.6615 - val_loss: 0.1620
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.7133 - loss: 0.1018 - val_accuracy: 0.6371 - val_loss: 0.1611
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7127 - loss: 0.1015 - val_accura

In [13]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.5842 - loss: 0.1612 - val_accuracy: 0.6189 - val_loss: 0.1735
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6848 - loss: 0.1199 - val_accuracy: 0.6897 - val_loss: 0.1351
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.7047 - loss: 0.1094 - val_accuracy: 0.7022 - val_loss: 0.1220
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7194 - loss: 0.1031 - val_accuracy: 0.7069 - val_loss: 0.1200
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.7329 - loss: 0.0967 - val_accuracy: 0.6907 - val_loss: 0.1416
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7411 - loss: 0.0914 - val_accuracy: 0.7146 - val_loss: 0.1074
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7499 - loss: 0.0880 - val_accuracy: 0.6500 - val_loss: 0.1362
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.7482 - loss: 0.0861 - val_accura

In [14]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 420.81 seconds
  Test Time: 0.72 seconds
  Precision: [0.73154362 0.13360324 0.47286822 0.44755245 0.91616039]
  Recall: [0.58133333 0.61111111 0.59223301 0.60952381 0.75174477]
  F1-Score: [0.64784547 0.2192691  0.52586207 0.51612903 0.82584885]
  Confusion Matrix:
[[218  83  25   8  41]
 [ 10  33   6   0   5]
 [ 15  11  61   6  10]
 [  3  18   7  64  13]
 [ 52 102  30  65 754]]
Week 2:
  Train Time: 431.38 seconds
  Test Time: 0.72 seconds
  Precision: [0.77346278 0.23602484 0.4109589  0.48201439 0.92429379]
  Recall: [0.63733333 0.7037037  0.58252427 0.63809524 0.81555334]
  F1-Score: [0.69883041 0.35348837 0.48192771 0.54918033 0.86652542]
  Confusion Matrix:
[[239  56  33   7  40]
 [  7  38   3   1   5]
 [ 15  17  60   4   7]
 [  1  11  11  67  15]
 [ 47  39  39  60 818]]
Week 3:
  Train Time: 428.46 seconds
  Test Time: 0.70 seconds
  Precision: [0.76454294 0.27433628 0.43421053 0.488      0.9336333 ]
  Recall: [0.736  